In [1]:
import pandas as pd
import os

data_path = os.path.join(os.getcwd(), '../data')
df = pd.read_csv(os.path.join(data_path, 'dataset.csv'))

In [34]:
aux = df[1:10]
aux

,Text,language
1,sebes joseph pereira thomas på eng the jesuit...,Swedish
2,ถนนเจริญกรุง อักษรโรมัน thanon charoen krung เ...,Thai
3,விசாகப்பட்டினம் தமிழ்ச்சங்கத்தை இந்துப் பத்திர...,Tamil
4,de spons behoort tot het geslacht haliclona en...,Dutch
5,エノが行きがかりでバスに乗ってしまい、気分が悪くなった際に助けるが、今すぐバスを降りたいと運...,Japanese
6,tsutinalar i̇ngilizce tsuutina kanadada albert...,Turkish
7,müller mox figura centralis circulorum doctoru...,Latin
8,برقی بار electric charge تمام زیرجوہری ذرات کی...,Urdu
9,シャーリー・フィールドは、サン・ベルナルド・アベニュー沿い市民センターとrtマーティン高校に...,Japanese


In [35]:
text = 'My name is X335. Nice to meet you! Today you turn 23 years old?'
text2 = 'エノが行きがかりでバスに乗ってしまい、気分が悪くなった際に助けるが、今すぐバスを降りたいと運'

In [36]:
import nltk
print(nltk.tokenize.word_tokenize(text))
print(nltk.tokenize.word_tokenize(text2))

['My', 'name', 'is', 'X335', '.', 'Nice', 'to', 'meet', 'you', '!', 'Today', 'you', 'turn', '23', 'years', 'old', '?']
['エノが行きがかりでバスに乗ってしまい、気分が悪くなった際に助けるが、今すぐバスを降りたいと運']


In [37]:
tokenizer = nltk.tokenize.RegexpTokenizer(r'\b[^\d\W]+\b')
print(tokenizer.tokenize(text.lower()))
print(tokenizer.tokenize(text2.lower()))

['my', 'name', 'is', 'nice', 'to', 'meet', 'you', 'today', 'you', 'turn', 'years', 'old']
['エノが行きがかりでバスに乗ってしまい', '気分が悪くなった際に助けるが', '今すぐバスを降りたいと運']


In [38]:
data = {'Text': ['My name is X335. Nice to meet you! Today you turn 23 years old?'], 'language': ['english']}
df = pd.DataFrame(data)
df

,Text,language
0,My name is X335. Nice to meet you! Today you t...,english


In [40]:
df.iloc[0,:]

Text        My name is X335. Nice to meet you! Today you t...
language                                              english
Name: 0, dtype: object

In [12]:
df["Text"].apply(lambda x : nltk.tokenize.sent_tokenize(x))#.apply(pd.Series,1).stack()

0    [My name is X335., Nice to meet you!, Today yo...
Name: Text, dtype: object

In [41]:
df['Text'] = df['Text'].apply(lambda x : nltk.tokenize.sent_tokenize(x))

In [45]:
df = df.explode('Text')

In [52]:
df.loc[:, 'Text']

0                My name is X335.
0               Nice to meet you!
0    Today you turn 23 years old?
Name: Text, dtype: object

In [7]:
def tokenize_sentence(sentence):
    import nltk
    
    tokenizer = nltk.tokenize.RegexpTokenizer(r'\b[^\d\W]+\b') # Splits text into the multiple non-numerical words contained in it
    sentence = tokenizer.tokenize(sentence)

    #porter_stem = nltk.stem.PorterStemmer() # Stems all the words using the Porter Stemmer algorithm
    sentence = ' '.join(sentence) # The generated list is joined again as a string as CountVectorizer cannot properly read it otherwise

    return sentence

def preprocess(sentence, labels):
    sentence = sentence.apply(tokenize_sentence) # Apply the auxiliary function to actually tokenize the sentences
    return sentence,labels

tokenize_sentence('test tes test')

'test tes test'

In [2]:
import unicodedata as ud

latin_letters= {}

def is_latin(uchr):
    try: return latin_letters[uchr]
    except KeyError:
         return latin_letters.setdefault(uchr, 'LATIN' in ud.name(uchr))

def is_greek(uchr):
    try: return latin_letters[uchr]
    except KeyError:
         return latin_letters.setdefault(uchr, 'GREEK' in ud.name(uchr))

def only_roman_chars(unistr):
    return all(is_latin(uchr)
           for uchr in unistr
           if uchr.isalpha())
    
only_roman_chars(u"ε")

False

In [43]:

import string

def tokenize_sentence(sentence):
    '''
    Input:
    Output:
    '''
    import nltk
    
    if contains_cjk(sentence):
        alph_is_cjk, idxs = alphabet_is_cjk(sentence)
        if alph_is_cjk:
            sentence = list(sentence)
            return sentence
        else:
            pass
            
    tokenizer = nltk.tokenize.RegexpTokenizer(r'\b[^\d\W]+\b') # Splits text into the multiple non-numerical words contained in it
    sentence = tokenizer.tokenize(sentence)

    porter_stem = nltk.stem.PorterStemmer() # Stems all the words using the Porter Stemmer algorithm
    sentence = ' '.join(porter_stem.stem(word) for word in sentence) # The generated list is joined again as a string as CountVectorizer cannot properly read it otherwise

    return sentence

def contains_cjk(unistr):
    '''
    Checks if a sentence contains chinese, japanese, or korean characters.
    '''
    import unicodedata as ud
    
    for uchr in unistr:
        if any(x in ud.name(uchr) for x in ['CJK', 'HANGUL']):
            return True
        else:
            continue
    return False

def contains_latin(unistr):
    '''
    Checks if a sentence contains latin characters.
    '''
    import unicodedata as ud
    
    for uchr in unistr:
        if any(x in ud.name(uchr) for x in ['LATIN']):
            return True
        else:
            continue
    return False

def alphabet_is_cjk(unistr):
    '''
    Checks if a sentence is mostly written in chinese, japanese, or korean characters.
    '''
    import unicodedata as ud
    
    cjk_chr = 0
    other_chr = 0
    idx_list = []
    for idx, uchr in enumerate(unistr):
        if any(x in ud.name(uchr) for x in ['CJK', 'HANGUL']):
            cjk_chr += 1
            idx_list.append(idx)
        else:
            other_chr += 1
            
    cjk_prop = cjk_chr/(other_chr+cjk_chr)
    if cjk_prop >= 0.5:
        return True, idx_list
    
    return False, idx_list

def alphabet_is_latin(unistr):
    '''
    Checks if a sentence is mostly written in latin characters.
    '''
    import unicodedata as ud
    
    latin_chr = 0
    other_chr = 0
    idx_list = []
    for idx, uchr in enumerate(unistr):
        if any(x in ud.name(uchr) for x in ['LATIN']):
            latin_chr += 1
            idx_list.append(idx)
        else:
            other_chr += 1
            
    latin_prop = latin_chr/(other_chr+latin_chr)
    if latin_prop >= 0.5:
        return True, idx_list
    
    return False, idx_list


In [17]:
def tokenize_sentence(sentence):
    '''
    Input:
    Output:
    '''
    import nltk
    
    if contains_cjk(sentence):
        alph_is_cjk, idxs = alphabet_is_cjk(sentence)
        if alph_is_cjk:
            final_sentence = ''
            for idx in idxs:
                final_sentence += sentence[idx] + ' '
            return final_sentence
        else:
            pass

    elif contains_latin(sentence):
        alph_is_latin, idxs = alphabet_is_latin(sentence)
        if alph_is_latin:
            final_sentence = ''
            for idx in idxs:
                final_sentence += sentence[idx]
            sentence = final_sentence
        else:
            pass

    tokenizer = nltk.tokenize.RegexpTokenizer(r'\b[^\d\W]+\b') # Splits text into the multiple non-numerical words contained in it
    sentence = tokenizer.tokenize(sentence)

    porter_stem = nltk.stem.PorterStemmer() # Stems all the words using the Porter Stemmer algorithm
    sentence = ' '.join(porter_stem.stem(word) for word in sentence) # The generated list is joined again as a string as CountVectorizer cannot properly read it otherwise

    return sentence

def contains_cjk(unistr):
    '''
    Checks if a sentence contains chinese, japanese, or korean characters.
    '''
    import unicodedata as ud
    
    for uchr in unistr:
        if any(x in ud.name(uchr) for x in ['CJK', 'HANGUL']):
            return True
        else:
            continue
    return False

def contains_latin(unistr):
    '''
    Checks if a sentence contains latin characters.
    '''
    import unicodedata as ud
    
    for uchr in unistr:
        if any(x in ud.name(uchr) for x in ['LATIN']):
            return True
        else:
            continue
    return False

def alphabet_is_cjk(unistr):
    '''
    Checks if a sentence is mostly written in chinese, japanese, or korean characters.
    '''
    import unicodedata as ud
    
    cjk_chr = 0
    other_chr = 0
    idx_list = []
    for idx, uchr in enumerate(unistr):
        if any(x in ud.name(uchr) for x in ['CJK', 'HANGUL']):
            cjk_chr += 1
            idx_list.append(idx)
        else:
            other_chr += 1
            
    cjk_prop = cjk_chr/(other_chr+cjk_chr)
    if cjk_prop >= 0.5:
        return True, idx_list
    
    return False, idx_list

def alphabet_is_latin(unistr):
    '''
    Checks if a sentence is mostly written in latin characters.
    '''
    import unicodedata as ud
    
    latin_chr = 0
    other_chr = 0
    idx_list = []
    for idx, uchr in enumerate(unistr):
        if any(x in ud.name(uchr) for x in ['LATIN']):
            latin_chr += 1
            idx_list.append(idx)
        elif uchr == ' ':
            idx_list.append(idx)
        else:
            other_chr += 1
            
    latin_prop = latin_chr/(other_chr+latin_chr)
    if latin_prop >= 0.5:
        return True, idx_list
    
    return False, idx_list

In [23]:
tokenize_sentence('a  aسعدت بلسعدت بلقائكسعدت بلقائكaaaaaaa')

'a aسعدت بلسعدت بلقائكسعدت بلقائكaaaaaaa'

In [43]:
ud.name('彈')

'CJK UNIFIED IDEOGRAPH-5F48'

In [18]:
ud.name('赛')

'CJK UNIFIED IDEOGRAPH-8D5B'

In [22]:
ud.name('ส')

'THAI CHARACTER SO SUA'

In [21]:
l = {}
l.setdefault('ې', 'ARABIC' in ud.name('ې'))

True